<a href="https://colab.research.google.com/github/sanha-hwang/OpenCV_practrice/blob/main/CNN%ED%95%A8%EC%88%98%EB%AA%A8%EC%9D%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Conv2D(
    filters, # int, 몇개의 컨볼루션 필터를 사용할건지, 피쳐맵 개수가 정해진다
    kernel_size, # int or tuple/list of 2 int , 컨볼루션 필터의 크기
    strides=(1, 1), # int or tuple/list of 2 int, stride크기 
    padding='valid', # 'valid' :padding=0 으로도 표기 패딩안씀, same: zero padding하여 차원 변화없게 함
    activation=None, 
    data_format=None, # 'channels_last' : batch_shape+(rows ,cols, channels) ,
                        'channels_first' : batch_shape+(channels, rows ,cols)
    
    dilation_rate=(1, 1), groups=1, 
    use_bias=True, kernel_initializer='glorot_uniform',
    bias_initializer='zeros', kernel_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    bias_constraint=None,
)

```





```
tf.keras.layers.MaxPool2D(
    pool_size=(2, 2), # int or tuple of 2 int, max value를 취함
    strides=None, # (default)pool_size
    padding='valid',
    data_format=None,
   
)
```





```
tf.keras.layers.Dropout(
    rate, # float, 0 ~ 1 값, 끄고 싶은 비율
    noise_shape=None,
    seed=None, 
)
```





```
tf.keras.layers.BatchNormalization(
    axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,
    beta_initializer='zeros', gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones', beta_regularizer=None,
    gamma_regularizer=None, beta_constraint=None, gamma_constraint=None, **kwargs
)
```



In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Input, Activation



In [8]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [9]:
from keras.utils.np_utils import to_categorical

X_train = X_train / 255 
X_test = X_test / 255

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

# converting y data into categorical (one-hot encoding)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(60000, 28, 28, 1) (10000, 28, 28, 1) (60000, 10) (10000, 10)


In [22]:

# CNN_base_model
input = Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
conv1 =Conv2D(filters = 64,
              kernel_size= (3,3),
              strides =(1,1),
              padding ='same', 
              activation='relu')(input)

pool_1 =  MaxPooling2D(pool_size= (2,2))(conv1)
x = Flatten()(pool_1)
dense_1 = Dense(64, activation='relu')(x)
output =  Dense(10, activation='softmax')(dense_1)
    
model = Model(inputs=input, outputs = output)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                802880    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 804,170
Trainable params: 804,170
Non-trainable params: 0
_______________________________________________________

In [25]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size= 32, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
1875/1875 [==============================] - 64s 34ms/step - loss: 0.1575 - accuracy: 0.9522 - val_loss: 0.0653 - val_accuracy: 0.9785
Epoch 2/10
1875/1875 [==============================] - 61s 32ms/step - loss: 0.0544 - accuracy: 0.9835 - val_loss: 0.0491 - val_accuracy: 0.9835
Epoch 3/10
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0357 - accuracy: 0.9891 - val_loss: 0.0546 - val_accuracy: 0.9824
Epoch 4/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0250 - accuracy: 0.9921 - val_loss: 0.0517 - val_accuracy: 0.9833
Epoch 5/10
1875/1875 [==============================] - 86s 46ms/step - loss: 0.0180 - accuracy: 0.9940 - val_loss: 0.0515 - val_accuracy: 0.9838
Epoch 6/10
1875/1875 [==============================] - 72s 39ms/step - loss: 0.0119 - accuracy: 0.9962 - val_loss: 0.0595 - val_accuracy: 0.9833
Epoch 7/10
1875/1875 [==============================] - 71s 38ms/step - loss: 0.0099 - accuracy: 0.9969 - val_loss: 0.0620 -

In [30]:
# CNN_advanced_model
input = Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
conv1 = Conv2D(filters = 64,
              kernel_size= (3,3),
              strides =(1,1),
              padding ='same')(input)
bn_1 = BatchNormalization()(conv1)
act_1 = Activation('relu')(bn_1)


pool_1 =  MaxPooling2D(pool_size= (2,2))(act_1)
x = Flatten()(pool_1)
dense_1 = Dense(64, activation='relu')(x)
drop_1 = Dropout(0.3)(dense_1)
output =  Dense(10, activation='softmax')(drop_1)
    
advanced_model = Model(inputs=input, outputs = output)

advanced_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8028

In [31]:
advanced_model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
advanced_model.fit(X_train, y_train, batch_size= 32, epochs=3, validation_data=(X_test,y_test))

Epoch 1/3
1875/1875 [==============================] - 112s 59ms/step - loss: 0.6066 - accuracy: 0.7954 - val_loss: 0.1749 - val_accuracy: 0.9506
Epoch 2/3
1875/1875 [==============================] - 104s 56ms/step - loss: 0.3937 - accuracy: 0.8658 - val_loss: 0.1104 - val_accuracy: 0.9686
Epoch 3/3
1875/1875 [==============================] - 109s 58ms/step - loss: 0.3027 - accuracy: 0.8931 - val_loss: 0.0990 - val_accuracy: 0.9693
